In [3]:
from tkinter import *
import PIL
from PIL import Image, ImageDraw
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import linear_model
from sklearn.metrics import r2_score
from collections import deque
from random import shuffle
from collections import Counter


### constants

# data capture framework
sampling_frequency = 1.0    # sampling frequency of the mouse
canvas_size = (640, 480)    # size in pixels of the canvas for collecting mouse data

# analysis framework
weights = np.array([
    [0.5, 0.5],            # weights for the Space features
    [0.1, 0.9]             # weights for the Time features
])

enable_continuous_analysis = 0     # dont' touch, its value is set automatically by the canvas
buffer_size = 55               # size of the fixed time window used for the continuous analysis only
character_changed = 0           # it indicates if the next character of the sequence has to be played
characters= ['cook','kangaroo','airplane','woodman'] #the characters of the game
count_buffers = 0
count = -1
count_attempts = 0


### global variables and data structures

buffer = deque([], buffer_size)   # used for continuous analysis only
how_many = 0                      # used for continuous analysis only
results = list()                  # used for continuous analysis only
lastx, lasty = None, None

def initialize():
    global buffer, how_many, results
          
    for i in range(1, buffer_size):
        buffer.append((0,0))
    how_many = 0
    results = []
    return

def append(point):
    global buffer, how_many
    buffer.appendleft(point)
    how_many = how_many + 1
    return

def append_result(data):
    global results
    results.append(data) 
    return

def who_is_mimicked(results):
    #FILL HERE!
    return(who_is)
 

def start_analysis():
    global enable_continuous_analysis, character_changed, count
    
    sequence_char = generate_characters_sequence()
    
    enable_continuous_analysis  = 1
    character_changed = 1
    count = -1
    return

def generate_characters_sequence():
    global characters
    #FILL HERE!
    return characters

### Laban analysis functions  

# Laban's Space
def get_distance(point1, point2):
    return np.linalg.norm(np.array(point2) - np.array(point1))

def path_length(points):
    length = 0
    for (i, j) in zip(points, points[1:]):
        length += get_distance(i, j)
    return length

def directness_index(points):
    #FILL HERE!
    return DI

def goodness_of_linear_fit(points):
    #FILL HERE!
    return r2

def laban_space(points, weights, canvas_size=(640, 480), display=0):
    s1 = directness_index(points)
    s2 = goodness_of_linear_fit(points)
    score = weights[0] * s1 + weights[1] * s2
    if (display == 1):
        print("\n--- Laban's Space features ---")
        print("Directness index =", s1)
        print("Goodness of fit =", s2)
        print("Total Space score =", score)
    return score

# Laban's Time
def speed(points, fs=1.0):
    dt = 1.0 / fs
    sp = []
    for (i, j) in zip(points, points[1:]):
        sp.append(np.linalg.norm([(j[0] - i[0]) / dt, (j[1] - i[1]) / dt]))
    return sp

def average_normalized_kinetic_energy(points, fs=1.0):
    sp = speed(points, fs)
    squares = np.square(sp)
    max_square = np.amax(squares)
    return np.mean(np.divide(squares, max_square))
        
def impulsiveness(points, fs=1.0):
    #FILL HERE!
    return IMP

def laban_time(points, weights, fs=1.0, display=0):
    t1 = average_normalized_kinetic_energy(points)
    t2 = impulsiveness(points)
    score = weights[0] * t1 + weights[1] * t2
    if (display == 1):
        print("\n--- Laban's Time features ---")
        print("Average normalized kinetic energy =", t1)
        print("Impulsiveness =", t2)
        print("Total Time score =", score)
    return score

# Laban's analysis
def laban_analysis(data, weights, canvas_size=(640, 480), fs=1.0, display=0):
    if len(data) != 0:
        s = laban_space(data, weights[0,:], canvas_size, display)
        t = laban_time(data, weights[1,:], sampling_frequency, display)
        return s, t
    else:
        return -1, -1
    
    
def analyse_continuous_data(): #MODIFY THIS FUNCTION!
    global count_buffers, results, count, characters, character_changed, count_attempts
    s, t = laban_analysis(list(buffer), weights)
    if (s == -1 and t == -1):
        print("Warning: no data to analyze.")
        return
    r = which_character(s, t)
  
    append_result(r)
    count_buffers = count_buffers +1
    
    if count_buffers == 5:
        char_mimicked = who_is_mimicked(results)
                
        print("who:", characters[count], "done:", char_mimicked)
        
        if char_mimicked == characters[count] 
            character_changed = 1
            delete_image()
          
                    
        count_buffers = 0
        results = []
        buffer.clear()
        print("END of the current attempt")
        
    return  


def which_character(s, t):
    if s < 0.5 and t < 0.45:
        r="cook"
    elif s > 0.45 and t > 0.45:
        r="airplane"
    elif s < 0.5 and t > 0.55:
        r="kangaroo"
    elif s >0.55 and t < 0.45:
        r="woodman"
    else:
        r="Try again, I do not understand who is mimicked!"
    return r 

def synthesis(ch): 
        #FILL HERE!
        root.destroy()
        
        return

       


### canvas and buttons handling functions

def create_widgets():
    deleteb = Button(cv, text = 'Delete', command = delete_image)
    deleteb.configure(width = 20)
    deleteb_window = cv.create_window(400, 450, anchor = CENTER, window=deleteb)
    
    startgameb=Button(cv, text = 'Start Game', command = start_analysis)
    startgameb.configure(width=20)
    startgameb_window = cv.create_window(250, 450, anchor = CENTER, window=startgameb)
    
    cv.bind('<1>', activate_paint)
    cv.pack(expand=YES, fill=BOTH)
    return

def activate_paint(e):
    global lastx, lasty
    cv.bind('<B1-Motion>', paint)
    lastx, lasty = e.x, e.y
    return

def paint(e):
    global lastx, lasty, count, character_changed
    x, y = e.x, e.y
    cv.create_line((lastx, lasty, x, y), width=1)
    draw.line((lastx, lasty, x, y), fill='black', width=1)
    lastx, lasty = x, y
    append((lastx, lasty)) 
    
    if character_changed == 1 and count <=3:
        count=count+1
        
        if count==4:
            synthesis(characters)
            return
        
        cv.create_text(500,10,fill="darkblue",font="Times 20 italic bold",text=characters[count])
        character_changed = 0
        
        print("Number of character in the sequence:",count+1)
        
       
        
    if how_many >= buffer_size and enable_continuous_analysis and (how_many % buffer_size*0.5)==0 and count <= 3:
        analyse_continuous_data()
        
      
    return
    
def delete_image():
    cv.delete("all")
    create_widgets()
    return


### main program

initialize()
root = Tk()
cv = Canvas(root, width=canvas_size[0], height=canvas_size[1], bg='white')
im = PIL.Image.new('RGB', canvas_size, 'white')
draw = ImageDraw.Draw(im)
create_widgets()
root.mainloop()


['airplane', 'woodman', 'cook', 'kangaroo']
Number of character in the sequence: 1
